<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/redrock_015_task2_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2_classification"
    name = "redrock_015_task2"

    # model
    pseudo_base_model_name = "ioai2024japan/redrock_015_task2_finetune"
    base_model_name = "ioai2024japan/redrock_005_task2_pretrain_wandb"
    tokenizer_name = "ioai2024japan/redrock_005_task2_tokenizer"
    num_classes = 5

    # training
    epochs = 40

    scheduler='linear' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 1e-05

    # dataset
    max_length = 256

    # T4: 32
    # L4: 64
    train_batch_size = 64
    eval_batch_size = 64

    seed=42
    train=True

    pseudo_size = 60000
    pseudo_select_size = 1500

# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [ ]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0s
  !pip install evaluate==0.4.2
  !pip install accelerate -U

if importlib.util.find_spec('wandb') is None:
  !pip install wandb -q

import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

import evaluate

import wandb

from huggingface_hub import login

wandb.login(key=userdata.get('wandb_token'))
login(token=read_access_token)

brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': '90'
}

def transliterate_brahmi_to_devanagari(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text

f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def pred_pseudo(model, train_loader, device):
    model.eval()
    predictions_list = []
    confidences_list = []
    for batch in tqdm(train_loader):
        batch = to_device(batch, device)
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        confidences = F.softmax(logits, dim=-1)
        pred_confidence,_ = confidences.max(dim=-1)
        #pred_confidence,_ = pred_confidence_temp.max(dim=-1)
        # print(pred_confidence.shape)
        predictions = torch.argmax(logits, dim=-1)
        #predictions = torch.mode(predictions, dim=0).values
        # print(predictions.shape)
        predictions_list.extend(predictions.cpu().numpy())
        confidences_list.extend(pred_confidence.cpu().numpy())

    return predictions_list, confidences_list

In [ ]:
def pseudo_get_data(raw_dataset, transform_raw, model, device):
    pseudo_size = CFG.pseudo_size

    raw_eval_batch = raw_dataset["train"].select(range(0, pseudo_size))

    print(raw_eval_batch)

    tokenized_eval_batch = raw_eval_batch.with_transform(transform_raw)

    raw_loader = DataLoader(
        tokenized_eval_batch,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )

    predictions, confidences = pred_pseudo(model, raw_loader, device)

    top_conf = np.argsort(confidences)[-CFG.pseudo_select_size:]

    selected_texts = [transliterate_brahmi_to_devanagari(raw_eval_batch[int(i)]["text"]) for i in top_conf]
    selected_labels = [predictions[int(i)] for i in top_conf]

    pseudo_labeled_dataset = Dataset.from_dict({
        'text': selected_texts,
        'label': selected_labels
    })

    return pseudo_labeled_dataset, confidences

In [ ]:
raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)
classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)

Generating train split:   0%|          | 0/611245 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1524 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/218 [00:00<?, ? examples/s]

In [ ]:
def main():

    wandb.init(
        name=CFG.name,
        project=CFG.project,
        config=cfg
    )

    pseudo_model = AutoModelForSequenceClassification.from_pretrained(
        CFG.pseudo_base_model_name, num_labels=CFG.num_classes
    ).cuda()

    model = AutoModelForSequenceClassification.from_pretrained(
        CFG.base_model_name, num_labels=CFG.num_classes
    ).cuda()

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)
    classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)

    tokenizer = AutoTokenizer.from_pretrained(CFG.tokenizer_name, token=read_access_token)

    def transform_raw(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        return inputs

    def transform(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        inputs["labels"] = example_batch["label"]
        return inputs

    pseudo_data, confidences = pseudo_get_data(raw_dataset, transform_raw, pseudo_model, device)
    pseudo_labeled_tokens = pseudo_data.with_transform(transform)

    tokenized_data = classification_dataset.with_transform(transform)

    train_dataset = tokenized_data["train"]
    eval_dataset = tokenized_data["dev"]

    combined_train_dataset = concatenate_datasets([pseudo_labeled_tokens, train_dataset])

    combined_train_loader = DataLoader(
        combined_train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )

    eval_loader = DataLoader(
        eval_dataset,
        batch_size=CFG.eval_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=False,
    )

    num_training_steps = CFG.epochs * len(combined_train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    def train_one_epoch(model, scheduler, train_loader, optimizer):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, dynamic_ncols=True)

        for step, batch in enumerate(progress_bar):
            batch = to_device(batch, device)
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )
            loss = outputs.loss
            loss.backward()

            text = f"step {step}, loss: {loss:.5f}"
            progress_bar.set_description(text)

            wandb.log(
                {
                    "train_loss": loss,
                    "lr": optimizer.param_groups[0]["lr"],
                    "step": step,
                }
            )
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

    def evaluate_model(model, test_loader):
        model.eval()
        predictions = []
        labels = []
        for batch in eval_loader:
            batch = to_device(batch, device)
            with torch.no_grad():
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    token_type_ids=batch["token_type_ids"],
                    labels=batch["labels"],
                )

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=-1)
            predictions.append(prediction.cpu().numpy())
            labels.append(batch["labels"].cpu().numpy())

        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        return f1.compute(predictions=predictions, references=labels, average='macro')

    # Train and evaluate the model
    model.to(device)
    for i in range(CFG.epochs):
        train_one_epoch(model, scheduler, combined_train_loader, optimizer)
        accuracy = evaluate_model(model, eval_loader)
        wandb.log(
            {
                "epoch": i+1,
                "accuracy": accuracy
            }
        )
        print(f'Epoch {i+1} {accuracy}')
    return model

In [ ]:
model = main()

wandb: Currently logged in as: asiatic-cheetah (asiatic-cheetah-a). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

...

Epoch 38 {'f1': 0.7508567553676858}


  0%|          | 0/47 [00:00<?, ?it/s]

Epoch 39 {'f1': 0.7567735602904797}


  0%|          | 0/47 [00:00<?, ?it/s]

Epoch 40 {'f1': 0.7556140194309849}


In [ ]:
model.push_to_hub(
    f"ioai2024japan/{CFG.name}",
    token=userdata.get('hf_write'), private=True
)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/redrock_015_task2/commit/d2790d96c718a5a963983bee08a09c58b11b630d', commit_message='Upload BertForSequenceClassification', commit_description='', oid='d2790d96c718a5a963983bee08a09c58b11b630d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()


In [ ]:
terminate_session()